# Προχωρημένα Θέματα Βάσεων Δεδομένων

**Ονοματεπώνυμο:** Κωνσταντίνος Διβριώτης

**ΑΜ:** 03114140

## Query 3: 

Χρησιμοποιώντας ως αναφορά τα δεδομένα της απογραφής 2010 για τον πληθυσμό και εκείνα της απογραφής του 2015 για το εισόδημα ανα νοικοκυριό, να υπολογίσετε για κάθε περιοχή του Los Angeles τα παρακάτω:
- Το μέσο ετήσιο εισόδημα ανά άτομο
- Την αναλογία συνολικού αριθμού εγκλημάτων ανά άτομο

In [1]:
from pyspark.sql import SparkSession
from sedona.spark import *

spark = SparkSession \
    .builder \
    .appName("CensusDataAnalysis") \
    .getOrCreate()

sedona = SedonaContext.create(spark)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1041,application_1732639283265_1008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
DATA_BUCKET = "s3://initial-notebook-data-bucket-dblab-905418150721"
GROUP_BUCKET = "s3://groups-bucket-dblab-905418150721/group15"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Διάβασμα και Επισκόπηση αρχείων εισόδου

In [3]:
from pyspark.sql.types import StructField, StructType, StringType
from pyspark.sql.functions import regexp_replace, col

income_schema = StructType([
    StructField("Zip Code", StringType()),
    StructField("Community", StringType()),
    StructField("Estimated Median Income", StringType())
])

income_data = spark.read.csv(f"{DATA_BUCKET}/LA_income_2015.csv", header=True, schema=income_schema)

# Μετατροπή του Estimated Median Income σε αριθμητική μορφή
income_data = income_data \
    .withColumn(
        "Estimated Median Income",
        regexp_replace(col("Estimated Median Income"), "[$,]", "").cast("float")
    ) \
    .select("Zip Code", "Estimated Median Income")

income_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------------+
|Zip Code|Estimated Median Income|
+--------+-----------------------+
|   90001|                33887.0|
|   90002|                30413.0|
|   90003|                30805.0|
|   90004|                40612.0|
|   90005|                31142.0|
+--------+-----------------------+
only showing top 5 rows

In [4]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType

# Ορισμός του schema των dataset
crimes_schema = StructType([
    StructField("DR_NO", StringType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", StringType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", StringType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", StringType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", DoubleType()),
    StructField("LON", DoubleType())
])

# Διαβάζουμε τα 2 datasets (2010-2019 και 2020-σήμερα) και τα συνενώνουμε σε 1
crime_data_2010_2019 = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv", header=True, schema=crimes_schema)
crime_data_2020_present = spark.read.csv(f"{DATA_BUCKET}/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv", header=True, schema=crimes_schema)
crime_data = crime_data_2010_2019.union(crime_data_2020_present)

# Μετατρέπουμε τις στήλες LAT, LON σε geometry με το ST_POINT
crime_data = crime_data \
                .withColumn("geom", ST_Point("LON", "LAT")) \
                .filter(col("geom") != ST_Point(0, 0)) \
                .select("DR_NO", "geom")

crime_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|    DR_NO|                geom|
+---------+--------------------+
|001307355|POINT (-118.2695 ...|
|011401303|POINT (-118.3962 ...|
|070309629|POINT (-118.2524 ...|
|090631215|POINT (-118.3295 ...|
|100100501|POINT (-118.2488 ...|
+---------+--------------------+
only showing top 5 rows

In [5]:
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true") \
            .load(f"{DATA_BUCKET}/2010_Census_Blocks.geojson") \
            .selectExpr("explode(features) as features") \
            .select("features.*")

blocks_data = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                    blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type") \
            .filter(col("COMM").isNotNull() & (col("POP_2010") > 0)) \
            .select("COMM", "ZCTA10", "POP_2010", "geometry")

blocks_data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+--------+--------------------+
|                COMM|ZCTA10|POP_2010|            geometry|
+--------------------+------+--------+--------------------+
|West Antelope Valley| 93243|       5|POLYGON ((-118.70...|
|           San Pedro| 90732|      69|POLYGON ((-118.31...|
|         Diamond Bar| 91789|      79|POLYGON ((-117.84...|
|           San Pedro| 90731|     120|POLYGON ((-118.28...|
|           San Pedro| 90731|     240|POLYGON ((-118.29...|
+--------------------+------+--------+--------------------+
only showing top 5 rows

In [6]:
blocks_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- COMM: string (nullable = true)
 |-- ZCTA10: string (nullable = true)
 |-- POP_2010: long (nullable = true)
 |-- geometry: geometry (nullable = true)

In [7]:
blocks_data_description_schema = StructType([
    StructField("field", StringType()),
    StructField("type", StringType()),
    StructField("meaning", StringType())
])

blocks_data_description = spark.read.csv(f"{DATA_BUCKET}/2010_Census_Blocks_fields.csv", header=True, schema=blocks_data_description_schema)

blocks_data_description \
        .filter(col("field").isin("COMM", "ZCTA10", "POP_2010", "geometry")) \
        .show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+-------------------------------------------------------------------------+
|field   |type    |meaning                                                                  |
+--------+--------+-------------------------------------------------------------------------+
|COMM    |string  |Unincorporated area community name and LA City neighborhood              |
|POP_2010|long    |Population (PL 94-171 Redistricting Data Summary File - Total Population)|
|ZCTA10  |string  |Zip Code Tabulation Area                                                 |
|geometry|geometry|Geometry of the block                                                    |
+--------+--------+-------------------------------------------------------------------------+

## Υλοποίηση με DataFrame

In [9]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Σύνδεση Εισοδήματος / Περιοχών με βάση το ZIP Code
income_per_block = blocks_data \
                        .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
                        .groupBy("COMM") \
                        .agg( \
                            sum("POP_2010").alias("Population"), \
                            sum("Estimated Median Income").alias("Total Income") \
                        )

# Σύνδεση Εγκλημάτων / Περιοχών με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής
crimes_per_block = crime_data \
                        .join(blocks_data, ST_Within(crime_data["geom"], blocks_data["geometry"]), "inner") \
                        .groupBy("COMM") \
                        .agg(count("*").alias("Total Crimes"))

# Aναλογία συνολικού αριθμού εγκλημάτων ανά άτομο
result = income_per_block \
                .join(crimes_per_block, on=["COMM"]) \
                .withColumn("Income per Person", col("Total Income") / col("Population")) \
                .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
                .select("COMM", "Income per Person", "Crimes per Person")

result.show()
end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|              COMM| Income per Person|   Crimes per Person|
+------------------+------------------+--------------------+
|        South Park| 119.3436899785335|  0.8312902496893007|
|   Exposition Park|159.45045784492265|  0.9741195783209788|
|           Central|135.04158432612502|  0.6675512393427814|
|       West Vernon|142.88127236580516|  1.0376739562624255|
|    Vernon Central|116.06856533595784|   0.663406532570585|
|Wholesale District|183.52424911185273|  1.6380665303046613|
|          Downtown|183.67133547731888|   2.974018280297901|
|   University Park|102.57116979309234|  0.7612511069962161|
|        Pico-Union|115.16710209003216|   0.706842845659164|
|        West Adams|136.08198018309912|  0.7499905813208756|
|   Wilshire Center| 92.47700602600698|  0.8269584522676816|
|    Temple-Beaudry|180.82272665116915|  0.6359086558184056|
|          Westlake| 50.96431055309492|  0.8648925656136754|
|       Pico Rivera| 456

In [10]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 32.24 seconds

In [11]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (36)
+- Project (35)
   +- SortMergeJoin Inner (34)
      :- Sort (15)
      :  +- HashAggregate (14)
      :     +- Exchange (13)
      :        +- HashAggregate (12)
      :           +- Project (11)
      :              +- BroadcastHashJoin Inner BuildRight (10)
      :                 :- Project (5)
      :                 :  +- Filter (4)
      :                 :     +- Generate (3)
      :                 :        +- Filter (2)
      :                 :           +- Scan geojson  (1)
      :                 +- BroadcastExchange (9)
      :                    +- Project (8)
      :                       +- Filter (7)
      :                          +- Scan csv  (6)
      +- Sort (33)
         +- HashAggregate (32)
            +- Exchange (31)
               +- HashAggregate (30)
                  +- Project (29)
                     +- RangeJoin (28)
                        :- Union (22)
                        :  :- Project (18)
           

Από προεπιλογή, το Spark χρησιμοποιεί τη στρατηγική **Sort Merge Join** (δηλαδή Merge).

Θα δοκιμάσουμε να αναγκάσουμε το Spark να χρησιμοποιήσει διαφορετικές στρατηγικές, ώστε να συγκρίνουμε την απόδοσή τους.

### 1. BROADCAST

In [27]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Σύνδεση Εισοδήματος / Περιοχών
income_per_block = blocks_data  \
                        .hint("BROADCAST") \
                        .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
                        .groupBy("COMM") \
                        .agg( \
                            sum("POP_2010").alias("Population"), \
                            sum("Estimated Median Income").alias("Total Income") \
                        )

# Σύνδεση Εγκλημάτων / Περιοχών
crimes_per_block = crime_data \
                        .hint("BROADCAST") \
                        .join(blocks_data, ST_Within(crime_data["geom"], blocks_data["geometry"]), "inner") \
                        .groupBy("COMM") \
                        .agg(count("*").alias("Total Crimes"))

# Aναλογία συνολικού αριθμού εγκλημάτων ανά άτομο
result = income_per_block \
                .hint("BROADCAST") \
                .join(crimes_per_block, on=["COMM"]) \
                .withColumn("Income per Person", col("Total Income") / col("Population")) \
                .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
                .select("COMM", "Income per Person", "Crimes per Person")

result.show()
end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://ec2-35-159-120-182.eu-central-1.compute.amazonaws.com:8998/sessions/914/statements/27 with error payload: {"msg":"requirement failed: Session isn't active."}


Το query δε μπορεί να εκτελεστεί με τη στρατηγική Broadcast στα join, λόγω του μεγάλου όγκου δεδομένων.

### 2. MERGE

In [13]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Σύνδεση Εισοδήματος / Περιοχών με βάση το ZIP Code
income_per_block = blocks_data  \
                        .hint("MERGE") \
                        .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
                        .groupBy("COMM") \
                        .agg( \
                            sum("POP_2010").alias("Population"), \
                            sum("Estimated Median Income").alias("Total Income") \
                        )

# Σύνδεση Εγκλημάτων / Περιοχών με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής
crimes_per_block = crime_data \
                        .hint("MERGE") \
                        .join(blocks_data, ST_Within(crime_data["geom"], blocks_data["geometry"]), "inner") \
                        .groupBy("COMM") \
                        .agg(count("*").alias("Total Crimes"))

# Aναλογία συνολικού αριθμού εγκλημάτων ανά άτομο
result = income_per_block \
                .hint("MERGE") \
                .join(crimes_per_block, on=["COMM"]) \
                .withColumn("Income per Person", col("Total Income") / col("Population")) \
                .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
                .select("COMM", "Income per Person", "Crimes per Person")

result.show()
end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+--------------------+
|                COMM| Income per Person|   Crimes per Person|
+--------------------+------------------+--------------------+
|                    | 649.5344341988502|1.543269416258343...|
|               Acton|3226.7076469141875|1.229407425620850...|
|     Adams-Normandie|160.62637082376943|  0.7148686559551135|
|        Agoura Hills| 787.9183472700443|2.951303492375799...|
|            Alhambra| 452.1307272924214|6.258349480677346E-4|
|              Alsace|186.29007503410642|  0.5416098226466576|
|            Altadena| 814.0934146854618|2.337704841386726...|
|            Anaverde|2450.7256874580817|0.002682763246143...|
|Angeles National ...| 4089.636842105263| 0.10300751879699248|
|    Angelino Heights| 241.8594276094276|  0.5989057239057239|
|             Arcadia|  592.959316784759|3.128274912799336...|
|              Arleta|312.64241391896826|  0.4264509064363061|
|      Athens Village| 364.7324789747697|0.002002402883

In [14]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 31.77 seconds

In [15]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (39)
+- Project (38)
   +- SortMergeJoin Inner (37)
      :- Sort (18)
      :  +- HashAggregate (17)
      :     +- Exchange (16)
      :        +- HashAggregate (15)
      :           +- Project (14)
      :              +- SortMergeJoin Inner (13)
      :                 :- Sort (7)
      :                 :  +- Exchange (6)
      :                 :     +- Project (5)
      :                 :        +- Filter (4)
      :                 :           +- Generate (3)
      :                 :              +- Filter (2)
      :                 :                 +- Scan geojson  (1)
      :                 +- Sort (12)
      :                    +- Exchange (11)
      :                       +- Project (10)
      :                          +- Filter (9)
      :                             +- Scan csv  (8)
      +- Sort (36)
         +- HashAggregate (35)
            +- Exchange (34)
               +- HashAggregate (33)
                  +- Project 

### 3. SHUFFLE HASH

In [17]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Σύνδεση Εισοδήματος / Περιοχών με βάση το ZIP Code
income_per_block = blocks_data  \
                        .hint("SHUFFLE_HASH") \
                        .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
                        .groupBy("COMM") \
                        .agg( \
                            sum("POP_2010").alias("Population"), \
                            sum("Estimated Median Income").alias("Total Income") \
                        )

# Σύνδεση Εγκλημάτων / Περιοχών με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής
crimes_per_block = crime_data \
                        .hint("SHUFFLE_HASH") \
                        .join(blocks_data, ST_Within(crime_data["geom"], blocks_data["geometry"]), "inner") \
                        .groupBy("COMM") \
                        .agg(count("*").alias("Total Crimes"))

# Aναλογία συνολικού αριθμού εγκλημάτων ανά άτομο
result = income_per_block \
                .hint("SHUFFLE_HASH") \
                .join(crimes_per_block, on=["COMM"]) \
                .withColumn("Income per Person", col("Total Income") / col("Population")) \
                .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
                .select("COMM", "Income per Person", "Crimes per Person")

result.show()
end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+--------------------+
|              COMM| Income per Person|   Crimes per Person|
+------------------+------------------+--------------------+
|    Vermont Knolls|208.44291881520567|  1.0672142942798897|
|     Vermont Vista|146.19928095872172|   1.256750998668442|
|      Harvard Park| 260.5980739155486|  1.0106455949735231|
|Florence-Firestone|213.46142986696086| 0.45668047232168735|
|       Pico Rivera| 456.6046995646786|3.177528518318452E-5|
|        Montebello|        395.805744|              9.6E-5|
|   Rowland Heights|384.75918600383346|4.078137106969536...|
|     Boyle Heights|  169.583309101483|  0.6253271299796452|
|           Maywood|167.33027924803795|3.650301149844862E-5|
|            Walnut| 656.5997531879884|6.855889208830385E-5|
|          Commerce| 501.5481556578024|2.339546128051158...|
|           Central|135.04158432612502|  0.6675512393427814|
|  East Los Angeles|258.15686367517316|0.002759067767131518|
|       Diamond Bar|501.

In [18]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 27.28 seconds

In [19]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (35)
+- Project (34)
   +- ShuffledHashJoin Inner BuildLeft (33)
      :- HashAggregate (15)
      :  +- Exchange (14)
      :     +- HashAggregate (13)
      :        +- Project (12)
      :           +- ShuffledHashJoin Inner BuildLeft (11)
      :              :- Exchange (6)
      :              :  +- Project (5)
      :              :     +- Filter (4)
      :              :        +- Generate (3)
      :              :           +- Filter (2)
      :              :              +- Scan geojson  (1)
      :              +- Exchange (10)
      :                 +- Project (9)
      :                    +- Filter (8)
      :                       +- Scan csv  (7)
      +- HashAggregate (32)
         +- Exchange (31)
            +- HashAggregate (30)
               +- Project (29)
                  +- RangeJoin (28)
                     :- Union (22)
                     :  :- Project (18)
                     :  :  +- Filter (17)
               

### 4. SHUFFLE REPLICATE NL

In [20]:
from pyspark.sql.functions import sum, count, col
import time

start_time = time.time()

# Σύνδεση Εισοδήματος / Περιοχών
income_per_block = blocks_data  \
                        .hint("SHUFFLE_REPLICATE_NL") \
                        .join(income_data, blocks_data["ZCTA10"] == income_data["Zip Code"]) \
                        .groupBy("COMM") \
                        .agg( \
                            sum("POP_2010").alias("Population"), \
                            sum("Estimated Median Income").alias("Total Income") \
                        )

# Σύνδεση Εγκλημάτων / Περιοχών με βάση το geometry, δηλαδή
# το POINT του εγκλήματος βρίσκεται εντός του POLYGON της περιοχής
crimes_per_block = crime_data \
                        .hint("SHUFFLE_REPLICATE_NL") \
                        .join(blocks_data, ST_Within(crime_data["geom"], blocks_data["geometry"]), "inner") \
                        .groupBy("COMM") \
                        .agg(count("*").alias("Total Crimes"))

# Aναλογία συνολικού αριθμού εγκλημάτων ανά άτομο
result = income_per_block \
                .hint("SHUFFLE_REPLICATE_NL") \
                .join(crimes_per_block, on=["COMM"]) \
                .withColumn("Income per Person", col("Total Income") / col("Population")) \
                .withColumn("Crimes per Person", col("Total Crimes") / col("Population")) \
                .select("COMM", "Income per Person", "Crimes per Person")

result.show()
end_time = time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+--------------------+
|                COMM| Income per Person|   Crimes per Person|
+--------------------+------------------+--------------------+
|         Culver City| 792.7922742586735| 0.00887277216264177|
|Rosewood/East Gar...|366.36426116838487| 0.08676975945017182|
|      Toluca Terrace| 260.9477325134512| 0.22213681783243658|
|        Elysian Park| 189.6804632618189|  0.6058477311562559|
|            Longwood|209.40380047505937|  0.7273159144893112|
|         Pico Rivera| 456.6046995646786|3.177528518318452E-5|
|              Malibu|1807.5768287860815|7.908264136022143E-5|
|       Green Meadows|246.80379395590535|  1.1079662983704153|
|    Cadillac-Corning|223.08117029257315|   0.581695423855964|
|          Montebello|        395.805744|              9.6E-5|
|            Mid-city|396.23837087663014|  0.7106492781923426|
|     Lincoln Heights|207.09009761109684|  0.5137105060364757|
|    Westlake Village| 835.2990326481257|1.209189842805

In [21]:
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 18.54 seconds

In [22]:
result.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (33)
+- Project (32)
   +- CartesianProduct Inner (31)
      :- HashAggregate (13)
      :  +- Exchange (12)
      :     +- HashAggregate (11)
      :        +- Project (10)
      :           +- CartesianProduct Inner (9)
      :              :- Project (5)
      :              :  +- Filter (4)
      :              :     +- Generate (3)
      :              :        +- Filter (2)
      :              :           +- Scan geojson  (1)
      :              +- Project (8)
      :                 +- Filter (7)
      :                    +- Scan csv  (6)
      +- HashAggregate (30)
         +- Exchange (29)
            +- HashAggregate (28)
               +- Project (27)
                  +- RangeJoin (26)
                     :- Union (20)
                     :  :- Project (16)
                     :  :  +- Filter (15)
                     :  :     +- Scan csv  (14)
                     :  +- Project (19)
                     :     +- Filter (18)
     

## Συμπεράσματα

Η στρατηγική join που πετυχαίνει την καλύτερη επίδοση είναι η **SHUFFLE REPLICATE NL** (*Shuffle Replicate Nested Loop*) με χρόνο 18.54 δευτερόλεπτα.

H στρατηγική **SHUFFLE HASH** πέτυχε λίγο καλύτερο χρόνο από τη **MERGE** με 27.28 και 31.77 δευτερόλεπτα αντίστοιχα.

Τέλος, η στρατηγική **BROADCAST** δε μπόρεσε να εκτελεστεί, καθώς εξαντλεί τους πόρους μνήμης. Αυτό συμβαίνει επειδή προσπαθεί να στείλει ολόκληρο το μικρότερο dataset σε όλους τους εκτελεστές (*executors*), για να εκτελεστεί το join τοπικά χωρίς shuffling. Έτσι η μέθοδος αποτυγχάνει, καθώς το μέγεθος του dataset που πρόκειται να μεταδοθεί (broadcast) είναι πολύ μεγάλο για να χωρέσει στη μνήμη των εκτελεστών.

Συνεπώς η καταλληλότερη στρατηγική για την περίπτωσή μας είναι η **SHUFFLE_REPLICATE_NL**.